In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df_train = pd.read_pickle("intermediate/train_df.pkl") 

In [ ]:
def auto_crop_black_borders(img, threshold=10):
    """
    Crop black borders from the right and bottom of an image.
    
    Parameters:
        img: Input image (NumPy array)
        threshold: Pixel intensity threshold to consider a pixel as "non-black"
    
    Returns:
        Cropped image (without black borders)
    """
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img

    # Create a binary mask of non-black pixels
    mask = gray > threshold

    # Find the bounding box of the non-black area
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)

    if not np.any(rows) or not np.any(cols):
        return img  # nothing to crop

    y_min, y_max = np.where(rows)[0][[0, -1]]
    x_min, x_max = np.where(cols)[0][[0, -1]]

    cropped = img[y_min:y_max+1, x_min:x_max+1]
    return cropped

In [ ]:
def split_by_color_dominance(df, path_col='path', threshold=0.25):
    good, bad = [], []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        path = row[path_col]
        img = cv2.imread(path)
        if img is None:
            continue

        img = auto_crop_black_borders(img)
        b, g, r = cv2.split(img)
        total = b.astype(np.float32) + g + r + 1e-5
        ratios = [np.mean(c / total) for c in (r, g, b)]

        (good if min(ratios) > threshold else bad).append(row)

    return pd.DataFrame(good), pd.DataFrame(bad)

In [ ]:
clean_df, removed_df = split_by_color_dominance(df_train)

In [ ]:
clean_df.to_pickle("intermediate/train_df_clean.pkl") 

In [ ]:
display(clean_df)

In [ ]:
display(removed_df) # A few (2 or 3) examples should not have been removed, but overall this looks okay

In [ ]:
equi = cv2.imread('images/argentina/1741691181_-22.1240522_-65.5582202.jpg')

img = cv2.cvtColor(equi, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB

plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
img = cv2.imread('images/france/1741690227_46.0110081_6.5302306.jpg')

b, g, r = cv2.split(img)
total = b.astype(np.float32) + g + r + 1e-5
ratios = [np.mean(c / total) for c in (r, g, b)]

print(ratios)

In [ ]:
img = cv2.imread('images/italy/1741690057_45.4795205_7.1412341.jpg')

b, g, r = cv2.split(img)
total = b.astype(np.float32) + g + r + 1e-5
ratios = [np.mean(c / total) for c in (r, g, b)]

print(ratios)

In [ ]:
def equirectangular_to_perspective(equi_img, fov, theta, size=512):
    """
    Convert equirectangular image to perspective (square output).

    Parameters:
        equi_img: Input equirectangular image
        fov: Horizontal field of view (degrees)
        theta: Yaw angle (degrees)
        size: Output image size (height = width = size)

    Returns:
        Perspective projection image
    """
    height, width = size, size
    equ_h, equ_w = equi_img.shape[:2]

    # Convert angles to radians
    fov_rad = np.deg2rad(fov)
    theta_rad = np.deg2rad(theta)

    # Grid of x, y in normalized view space
    x = np.linspace(-np.tan(fov_rad / 2), np.tan(fov_rad / 2), width)
    y = np.linspace(-1, 1, height)  # keep vertical stretch simple
    x, y = np.meshgrid(x, -y)  # flip y for image orientation
    z = np.ones_like(x)

    # Normalize direction vectors
    norm = np.sqrt(x**2 + y**2 + z**2)
    x /= norm
    y /= norm
    z /= norm

    # Rotate around Y axis (theta)
    x_rot = np.cos(theta_rad) * x + np.sin(theta_rad) * z
    z_rot = -np.sin(theta_rad) * x + np.cos(theta_rad) * z

    # Convert to spherical coordinates
    lon = np.arctan2(x_rot, z_rot)
    lat = np.arcsin(y)

    # Map to image coordinates
    u = (lon + np.pi) / (2 * np.pi) * equ_w
    v = (np.pi / 2 - lat) / np.pi * equ_h

    # Remap
    u = u.astype(np.float32)
    v = v.astype(np.float32)
    perspective = cv2.remap(equi_img, u, v, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)

    return perspective

In [ ]:
clean_df = pd.read_pickle("intermediate/train_df_clean.pkl")  

In [ ]:
def slice_square_images(image_path, size = 512):
    img = cv2.imread(image_path)
    img = auto_crop_black_borders(img)  # Remove black borders if needed

    views = ['front', 'right', 'back', 'left']

    image_name = image_path.split('/')[2].replace('.jpg', '')
    country = image_path.split('/')[1]
    
    output_folder = 'train_images_square/' + country + '/'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i in range(4):  
        view = equirectangular_to_perspective(img, fov=90, theta=i*90, size = size)

        # plt.imshow(view)
        # plt.axis('off')
        # plt.show()
        
        new_image_path = os.path.join(output_folder + f"{image_name}_{views[i]}.jpg")
        cv2.imwrite(new_image_path, view)

In [ ]:
slice_square_images('images/argentina/1741691181_-22.1240522_-65.5582202.jpg')

In [ ]:
tqdm.pandas() # to see a progress bar

clean_df['path'].progress_apply(slice_square_images)